# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Get the Wiki page as html text

In [1]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Using *BeautifulSoup* we can parse thew html and find the table 

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Now parse the HTML table to pandas dataframe

In [3]:

import pandas as pd
 
df = pd.read_html(website_url)[0]

# define the first row as header
df.columns = df.iloc[0]

# remove the first row since it is now the header
df = df.reindex(df.index.drop(0))
df.head(10)


,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


Remove the rows that contains 'Not assigned'

In [4]:
df = df[df != 'Not assigned']
df = df.dropna()
df.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern
15,M3B,North York,Don Mills North


combine the rows with the sam postcode

In [5]:
test  = df.groupby('Postcode')['Neighbourhood'].apply(', '.join).reset_index()
print(type(test))
test.head()


<class 'pandas.core.frame.DataFrame'>


,Postcode,Neighbourhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


Print the number of rows of the dataframe

In [6]:
test.shape

(102, 2)

In [8]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
comp_df = pd.merge(test, geo_df, how='right', left_on='Postcode', right_on='Postal Code',
         validate='one_to_one')
comp_df.head()

,Postcode,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Woburn,M1G,43.770992,-79.216917
4,M1H,Cedarbrae,M1H,43.773136,-79.239476


In [17]:

# Install and import required libraries, as in New Your clustering notebook
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
!conda install -c conda-forge beautifulsoup4 --yes 
!conda install -c conda-forge geocoder --yes 


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 483.65 kB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00 762.81 kB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.05 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  32.91 MB/s
vincent-0.4.4- 100% |###################

In [47]:
print(comp_df.columns.tolist())
comp_df = comp_df.dropna()

plod_df = comp_df[comp_df['Neighbourhood'].str.contains("Toronto")]
#comp_df.reindex()
plod_df.head()
#comp_df['Postcode']

plod_df.iloc[0]['Latitude']
plod_df.iloc[0]['Longitude']


['Postcode', 'Neighbourhood', 'Postal Code', 'Latitude', 'Longitude']


43.737473200000004

In [49]:
# We'll make a folium map of the neighborhoods. To do that, we need the address of Toronto, Ontario
torontoLat = plod_df.iloc[0]['Latitude']
torontoLon = plod_df.iloc[0]['Longitude']

map_toronto = folium.Map(location=[torontoLat, torontoLon], zoom_start=11)

# add markers to map
for lat, lng, label in zip(plod_df['Latitude'], plod_df['Longitude'], plod_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [52]:
CLIENT_ID = 'G0XJ2XFI1IPZMUR52JRVDEA31QO4DTXKTTIQ1DMOGM5EN0PU' # your Foursquare ID
CLIENT_SECRET = 'MZYX1FDUHTBPLNNHQBC4IZVPPHDB44GVAK2RQQDKHRIVM5JR' # your Foursquare Secret

In [53]:
VERSION = '20180605' # Foursquare API version
LIMIT=100

# This is copied from the New York lab
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(plod_df['Neighbourhood'], plod_df['Latitude'], plod_df['Longitude'])

CFB Toronto, Downsview East
East Toronto
North Toronto West
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Harbord, University of Toronto
Humber Bay Shores, Mimico South, New Toronto


In [54]:
print(toronto_venues.shape)
toronto_venues.head()

(273, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CFB Toronto, Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
1,"CFB Toronto, Downsview East",43.737473,-79.464763,Ttc Bus #120 - Plewes Rd,43.734898,-79.464221,Bus Stop
2,"CFB Toronto, Downsview East",43.737473,-79.464763,Ancaster Park,43.733724,-79.465528,Park
3,East Toronto,43.685347,-79.338106,TTC Stop #973,43.683198,-79.338428,Bus Stop
4,East Toronto,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park


In [55]:

# See number of venus by neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"CFB Toronto, Downsview East",3,3,3,3,3,3
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
East Toronto,5,5,5,5,5,5
"Harbord, University of Toronto",32,32,32,32,32,32
"Harbourfront East, Toronto Islands, Union Station",100,100,100,100,100,100
"Humber Bay Shores, Mimico South, New Toronto",14,14,14,14,14,14
North Toronto West,19,19,19,19,19,19


In [56]:
toronto_venues.groupby('Neighbourhood').count().shape


(7, 6)